Titanic gemisinde sınıf, cinsiyet ve yaş gibi faktörleri inceleyerek, kimlerin hayatta kaldığını inceleyeceğiz.

PassengerId: Yolcu ID
Survived: Hayatta kalma (0 = hayır; 1 = evet)
Pclass: Yolcu sınıfı (1 = birinci; 2 = ikinci; 3 = üçüncü)
Name: İsim
Sex: Cinsiyet
Age: Yaş
Sibsp: Gemideki eş kardeş sayısı
Parch: Gemideki ebeveyn / çocuk sayısı
Ticket: Bilet numarası
Fare: Yolcu ücreti
Cabin: Kabin
Embarked: Gemiye biniş limanı (C = Cherbourg; Q = Queenstown; S = Southampton)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import normalize
import warnings

In [ ]:
dizi = pd.read_csv('../input/titaniccsv/titanic.csv', header = 0)
dizi.info()

In [ ]:
dizi.head()

In [ ]:
dizi.describe()

In [ ]:
#Gözlem ve değişken sayılarını inceliyoruz.
dizi.shape

In [ ]:
#Her bir değişkenin veri tiplerini inceliyoruz.
dizi.dtypes

In [ ]:
##Şimdi her kolonda kaç tane eksik değer var onları inceliyoruz.
dizi.isnull().sum()

Eksik değerlerin yerini görselleştiriyoruz.

In [ ]:
fig, ax = plt.subplots(figsize = (9, 5))
sns.heatmap(dizi.isnull(), cmap = "cubehelix_r", yticklabels='')
plt.show()

In [ ]:
# Yaş sütununda boş olan değerleri cinsiyete göre dolduruyoruz. (Ortalama cinsiyet yaşına göre)
dizi['Age'] = dizi.groupby(['Sex'])['Age'].apply(lambda x: x.fillna(x.median()))
dizi.isnull().sum()
#Ortalama aldığımız zaman aykırı değerlerden etkilenebiliyor. Medyan, örneklemi daha iyi yansıttığı için alıyoruz.

In [ ]:
fig, ax = plt.subplots(figsize = (9, 5))
sns.heatmap(dizi.isnull(), cmap = "cubehelix_r", yticklabels='')
plt.show()

In [ ]:
#Çok fazla eksik olan Cabin kolonunu çıkartıyoruz. %50'den fazla eksik değer olduğu için.
dizi = dizi.drop(['Cabin'], axis=1)

In [ ]:
yasGoreSiralama = pd.cut(dizi["Age"], np.arange(0, 81, 10))
yasGrup = dizi.groupby(yasGoreSiralama).count()
yasGrup['Survived'].plot.bar()
plt.ylabel('Yolcu Sayısı')
plt.xlabel('Yaş Aralığı')
plt.show()

In [ ]:
#Dünya Sağlık Örgütü‘ne göre yaş aralığı bu şekilde:
#0-17 yaş arası: Ergen yaş aralığı.
#18-65 yaş arası: Genç yaş aralığı.
#66-79 yaş arası: Orta yaş aralığı
#80-99 yaş arası: Yaşlı
def yas_dagilimi(x):
    if  x <=17:
        return 'Ergen'
    elif x >= 18 and x <= 65:
        return 'Genç'
    elif x >= 66 and x <= 79:
        return 'Orta yaş'
    else:
        return 'Yaşlı'

In [ ]:
#Farklı yaş aralıklarına göre yolcuların dağılımı
plt.figure(figsize=(40,10))
dizi['Age'].apply(yas_dagilimi).value_counts().plot(kind='pie', autopct='%3.2f%%',fontsize=14)
plt.title('Farklı yaş aralıklarına göre yolcuların dağılımı')
plt.ylabel('Yaş Aralıkları')
plt.show()

In [ ]:
print('Hayatta kalma ile yaş arasındaki ilişkiyi kontrol ediyoruz. :\n',dizi['Survived'].corr(dizi['Age']))
#Hayatta kalma ile yaş arasında belirgin bir ilişki yoktur.

In [ ]:
dizi['Embarked'].describe()

In [ ]:
#Embarked sütunundaki eksik 2 hücreyi en fazla yolcunun biniş yaptığı S limanı ile dolduruyoruz.
dizi['Embarked'] = dizi['Embarked'].fillna('S')

In [ ]:
dizi['Embarked'].value_counts().plot(kind='bar')
plt.title('Yolcuların gemiye biniş limanlarına göre dağılımları ')
plt.ylabel('Sıklık')
plt.xlabel('S=Southampton, C=Cherbourg, Q=Queenstown')
plt.show()

In [ ]:
# Yolcu sınıfına göre ortalamaları inceliyoruz.
dizi.groupby('Pclass').mean()

In [ ]:
#Yolcu sınıfına göre hayatta kalma yolcu sayılarını buluyoruz.
sinifHayattaKalma = dizi.groupby(['Pclass', 'Survived'])
sinifHayattaKalma.size().unstack().plot(kind='bar', stacked=False)
plt.xlabel('Yolcu Sınıfı')
plt.ylabel('Sıklık')
plt.title('Yolcu sınıflarına göre hayatını kaybeden ve hayatta kalan yolcu sayıları görselleştirilmiştir.')
plt.show()

In [ ]:
plt.figure(figsize=(14,6))
sns.barplot(data=dizi, x="Pclass", hue='Sex', y='Survived', estimator=np.mean);
plt.xlabel('Yolcu sınıfı')
plt.ylabel('Cinsiyete göre hayatta kalma oranı')
sns.despine(top=True, right=True, left=False, bottom=False);

In [ ]:
cinsiyeteGoreDagilim = sns.barplot(x="Sex", y="Survived", hue="Sex", data=dizi);
cinsiyeteGoreDagilim.set_title("Cinsiyete Göre Hayatta Kalma Dağılımı");
plt.xlabel('Cinsiyet')
plt.ylabel('Hayatta Kalma Oranı')

In [ ]:
cinsiyetHayattaKalma = dizi.groupby(['Sex', 'Survived'])
cinsiyetHayattaKalma.size().unstack().plot(kind='bar',stacked=False)
plt.ylabel('Sıklık')
plt.xlabel('Cinsiyet')
plt.title('Cinsiyete göre hayatta kalma')
plt.show()

In [ ]:
#Hem sınıf, hemde cinsiyete göre istatistiğimizi inceliyoruz.
dizi.groupby(['Pclass','Sex']).mean()

Tüm sınıflarda kadınların hayatta kalma olasılığı erkeklerden çok daha fazla. 

In [ ]:
dizi['Pclass'].value_counts().plot(kind='barh', color='green', figsize=[16,4])
plt.xlabel('Sıklık')
plt.ylabel('Yolcu sınıfı')
plt.title('Yolcu sınıflarına göre yolcu sayılarının dağılımı')
plt.show()

In [ ]:
gemideKardesiOlmayanlar = dizi[dizi['SibSp'] == 0]
yalnizYolcular = gemideKardesiOlmayanlar[gemideKardesiOlmayanlar['Parch'] == 0]
yalnizYolcular.describe()

In [ ]:
#Yalnız olan yolcuları yaşlarına göre hayatta kalıp, kalmadıklarını inceliyoruz. 
def hayatta_kalma(x):
    if  x == 0:
        return 'Hayatını kaydeden'
    else:
        return 'Hayatta kalan'
plt.figure(figsize=(40,11))
yalnizYolcular['Survived'].apply(hayatta_kalma).value_counts().plot(kind='pie', autopct='%1.2f%%',fontsize=14)
plt.title('Yalnız olan yolcuların hayatta kalma oranları')
plt.ylabel('Hayatta kalma oranı')
plt.show()

In [ ]:
#Ailesiyle birlikte yolculuk yapanları, yalnız yolculuk yapan tüm yolculardan çıkartarak elde ediyoruz.
aileliYolcular = dizi.drop(yalnizYolcular.index[:])
aileliYolcular.describe()

In [ ]:
plt.figure(figsize=(40,11))
aileliYolcular['Survived'].apply(hayatta_kalma).value_counts().plot(kind='pie', autopct='%1.2f%%',fontsize=14)
plt.title('Aileli olan yolcuların hayatta kalma oranları')
plt.ylabel('Hayatta kalma oranı')
plt.show()

In [ ]:
dizi[["SibSp","Survived"]].groupby(["SibSp"]).mean().sort_values(by = 'SibSp',ascending = True)

Analize göre yanında bulunan insan sayısı arttıkça hayatta kalma oranı düşüyor.

In [ ]:
#Yolcuları yaş ortalamalarına göre ayırıyoruz.
def yas_dagilimi_sayisal(x):
    if  x <=17:
        return '0'
    elif x >= 18 and x <= 65:
        return '1'
    elif x >= 66 and x <= 79:
        return '2'
    else:
        return '3'

dizi['AgeAverage'] = dizi['Age'].apply(yas_dagilimi_sayisal)
yasAraligiDizi = dizi.copy()
#Yaşa göre ergen = 0, genc = 1, orta yaş = 2, yaşlı = 3 olarak atama yaptık.

In [ ]:
#Yaşa ve yaş ortalamasına göre aykırı değerleri inceliyoruz.
yasAraligiDizi = yasAraligiDizi.drop(['PassengerId','Survived','Pclass','Name','Sex','SibSp','Parch','Ticket','Fare','Embarked'], axis=1)

In [ ]:
yasAraligiDizi.hist(['Age','AgeAverage'], figsize=(15,5))
plt.ylabel('Hayatta kalma oranı')
plt.xlabel('Yaş')
plt.show()

In [ ]:
yeniDizi = dizi.copy()
yeniDizi.drop(['Name','Ticket','PassengerId','Name'], 1, inplace =True)
yeniDizi.Sex.replace(('male','female'), (0,1), inplace = True)
yeniDizi.Embarked.replace(('S','C','Q'), (0,1,2), inplace = True)

In [ ]:
#Sayısal değerler ile hayatta kalma arasındaki korelasyon matrisi
g = sns.heatmap(yeniDizi[["Survived","SibSp","Parch","Age","Fare"]].corr(),
                annot=True, fmt = ".2f", cmap = "coolwarm")

In [ ]:
#Gemideki ebeveyn / çocuk sayısı özelliğini, hayatta kalma oranına göre keşfediyoruz.
g  = sns.catplot(x="Parch",y="Survived",data=yeniDizi,kind="bar", height = 6, palette = "muted")
g.despine(left=True)
g.set_xlabels("Gemideki ebeveyn / çocuk sayısı")
g = g.set_ylabels("Hayatta Kalma Olasılığı")

In [ ]:
#Yaşa karşı cinsiyet, Gemideki ebeveyn / çocuk sayısı, yolcu sınıfı ve gemideki eş kardeş sayısını keşfetme.
g = sns.catplot(y="Age",x="Sex",data=yeniDizi,kind="box")
g = g.set_ylabels("Yaş")
g = g.set_xlabels("Cinsiyet")
g = sns.catplot(y="Age",x="Sex",hue="Pclass", data=yeniDizi,kind="box")
g = g.set_ylabels("Yaş")
g = g.set_xlabels("Cinsiyet")
g = sns.catplot(y="Age",x="Parch", data=yeniDizi,kind="box")
g = g.set_ylabels("Yaş")
g = g.set_xlabels("Gemideki ebeveyn / çocuk sayısı")
g = sns.catplot(y="Age",x="SibSp", data=yeniDizi,kind="box")
g = g.set_ylabels("Yaş")
g = g.set_xlabels("Gemideki eş kardeş sayısı")